# Ejercicio OpenStreetMap
Para este ejercicio vas a trabar con Overpass, que es la API que utiliza los datos de OpenStreetMap. Tras obtener datos geográficos mediante la API, tendrás que representarlos con Plotly.

Hay que obtener los siguientes datos de cada ubicación conseguida en OpenStreetMap
* tipo de amenity
* latitud
* longitud
* nombre
* direccion

**NOTA**: En este Notebook utilizarás gráficas de Plotly, y JupyterLab de momento no lo soporta, a no ser que instales una extensión. Para solucionar esto, abre el Notebook desde Jupyter normal:

![imagen](./img/jupyter.png)

Tienes en [este enlace](https://wiki.openstreetmap.org/wiki/Key:amenity#Healthcare) la lista de *amenities* que vas a usar en el notebook.

In [1]:
#!pip install plotly

## Instalación
Si no tienes instalado Plotly, abre un terminal y escribe:
```
pip install plotly
```

Importa las librerias que vamos a usar en el Notebook

In [2]:
import requests
import json
import pandas as pd

Ataca a la API de OpenStreetmap con el siguiente código de ejemplo

In [3]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="ice_cream"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()

<Response [200]>


Con el siguiente código podrás adaptar el JSON que devuelve la API, a un formato DataFrame más amigable

In [4]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

Utilizarás esta función para representar los datos en un mapa de Matplotlib

In [5]:
import plotly.express as px

mapbox_access_token = "xxxx"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

Como ejemplo, si quiero ver cuántos restaurantes tengo cerca de The Bridge:

In [6]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="restaurant"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,restaurant,40.423086,-3.693547,La Castafiore,NO STREET 9999
1,restaurant,40.422607,-3.693256,Sake Dining Himawari,NO STREET 9999
2,restaurant,40.423315,-3.692506,Ainhoa,Calle de Doña Bárbara de Braganza 12
3,restaurant,40.421960,-3.694270,La Kitchen,NO STREET 9999
4,restaurant,40.422427,-3.693028,Regañadientes,Calle del Almirante 24
5,restaurant,40.421951,-3.693535,Tampu,Calle de Prim 13
6,restaurant,40.423132,-3.694039,Taqueria La Lupita,Calle del Conde de Xiquena 9999
7,restaurant,40.423345,-3.692632,El Gato Persa,Calle de Doña Bárbara de Braganza 10
8,restaurant,40.423386,-3.694036,Shuwa Shuwa,Calle del Conde de Xiquena 12
9,restaurant,40.421872,-3.690240,Banzai,NO STREET 9999


In [7]:
pinta_mapa(df)

## Ejercicio 1
### Búsqueda de aparcamiento
Para este caso de uso imagina que vas a Madrid de viaje y estás buscando posibles parkings donde dejar el coche, crea un DataFrame con todos los parkings (de OpenStreetMap) de la ciudad de Madrid. Para ello define una cuadrícula con dos coordenadas sobre Madrid.

Representa en un mapa los resultados

In [8]:
overpass_query = """
[out:json];
node["amenity"="parking"]
  (40.396013, -3.746999, 40.480011, -3.625910); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,parking,40.426712,-3.714370,Princesa 25,NO STREET 9999
1,parking,40.414469,-3.703549,Parking Benavente,NO STREET 9999
2,parking,40.406444,-3.708171,NO NAME,NO STREET 9999
3,parking,40.447447,-3.627856,NO NAME,NO STREET 9999
4,parking,40.425182,-3.712927,Princesa,NO STREET 9999
...,...,...,...,...,...
239,parking,40.399212,-3.690897,NO NAME,NO STREET 9999
240,parking,40.399082,-3.691076,NO NAME,NO STREET 9999
241,parking,40.459477,-3.686302,Parking Mundial 24,Avenida de Alberto Alcocer 11
242,parking,40.453936,-3.702721,Edificio Tecnus,NO STREET 9999


In [9]:
pinta_mapa(df)

## Mudanza
Imagina que te quieres mudar al Berrueco, pero antes te gustaría obtener algún indicador de si hay civilización por esa zona. Mira a ver si existe gasolineras y/o colegios por la zona. Define, al igual que en el caso anterior, un rectángulo mediante dos coordenadas. Busca la manera de realizar una única petición a la API.

Representa después en un mapa, por color los colegios y gasolineras

In [10]:
overpass_query = """
[out:json];
node["amenity"="fuel"]
  (40.851838, -3.627774, 40.941457, -3.481755);
out;
node["amenity"="school"]
  (40.851838, -3.627774, 40.941457, -3.481755);
out;
"""

In [11]:
overpass_query = """
[out:json];
node["amenity"~"fuel|shool"]
  (40.851838, -3.627774, 40.941457, -3.481755);
out;
"""

In [12]:
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,fuel,40.905376,-3.607588,Repsol,NO STREET 9999
1,fuel,40.906677,-3.610172,Shell,CTRA. N-I KM 64.650 9999
2,fuel,40.856872,-3.612894,Shell,"AVDA. GENERALISIMO,12 9999"
3,fuel,40.875833,-3.602389,Shell,Avenida de La Cabrera 9999


In [13]:
pinta_mapa(df)

## Zonas COVID
Imagina que trabajas en el ayuntamiento de tu barrio/pueblo/ciudad, y por motivos del COVID tienes que identificar:
1. Todas las zonas abiertas de uso común (zonas de barbacoas, fuentes de agua y parques infantiles (kindergarten)), para cerrarlas.
2. Todos los hospitales y clínicas de la zona.

Como no sabes muy bien qué puntos serán, utilizas la API de OpenStreetMap. Representa todo en un único mapa

In [14]:
overpass_query = """
[out:json];
node["amenity"="bbq"]
  (40.410505, -3.715351, 40.448860, -3.677615);
out;
node["amenity"="drinking_water"]
  (40.410505, -3.715351, 40.448860, -3.677615);
out;
node["amenity"="hospital"]
  (40.410505, -3.715351, 40.448860, -3.677615);
out;
node["amenity"="clinic"]
  (40.410505, -3.715351, 40.448860, -3.677615);
out;
node["amenity"="kindergarten"]
  (40.410505, -3.715351, 40.448860, -3.677615);
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,drinking_water,40.413707,-3.695723,NO NAME,NO STREET 9999
1,drinking_water,40.412405,-3.691392,NO NAME,NO STREET 9999
2,drinking_water,40.413810,-3.709083,NO NAME,NO STREET 9999
3,drinking_water,40.444679,-3.708382,NO NAME,NO STREET 9999
4,drinking_water,40.439450,-3.691385,NO NAME,NO STREET 9999
...,...,...,...,...,...
248,kindergarten,40.447935,-3.704028,Chiquitín Cuatro Caminos,NO STREET 9999
249,kindergarten,40.434118,-3.709672,Escuela infantil municipal Fernando el Católico,Calle de Fernando el Católico 16B
250,kindergarten,40.424438,-3.706078,Family Moon,Calle de Minas 6
251,kindergarten,40.433793,-3.695049,St. Alice's nursery,Calle de Fernández de la Hoz 21


In [15]:
pinta_mapa(df)

## Supermercados
Para este ejemplo supondrás que trabajas en una cadena de supermercados en la que se está planteando construir un nuevo local en la provincia de Sevilla. Busca mediante la API de Overpass todos los supermercados de la provincia para poder comprobar en qué nucleos urbanos habría poca competencia.

En este caso hay que buscar por área, no por cuadrícula! Investiga en la documentación/Stackoverflow cómo se haría eso.

In [16]:
overpass_query = """
[out:json];
area[name="Sevilla"];
nwr[shop=supermarket](area);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,None,37.373002,-6.047204,MAS,NO STREET 9999
1,None,37.390016,-5.973769,Maxi Dia,NO STREET 9999
2,None,36.994532,-5.572320,Dia,NO STREET 9999
3,None,37.406364,-5.976342,El Jamón,NO STREET 9999
4,None,37.406508,-5.973342,Valdejimena,Avenida de San Juan de la Salle 9999
...,...,...,...,...,...
535,None,NaN,NaN,Superco,Calle Alfonso Braojos Garrido 9999
536,None,NaN,NaN,Cash Fresh,NO STREET 9999
537,None,NaN,NaN,NO NAME,NO STREET 9999
538,None,NaN,NaN,Dia,NO STREET 9999


In [17]:
df['tipo'] = df['tipo'].fillna("supermarket")
df = df.dropna()

In [18]:
pinta_mapa(df)

## Etiqueta en el mapa
Busca la manera de modificar la función `pinta_mapa()`, de tal manera que cuando pases el ratón por encima de cada punto, aparezca el nombre de cada nodo. Por ejemplo, que en el último apartado aparezcan los nombres de los supermercados.

In [19]:
def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            text = "name",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()
pinta_mapa(df)